In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('analysis_avgx')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

In [2]:
path=f'pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

# conn

In [3]:
key2tf2ratio={}
for ens in enss:
    key2tf2ratio[(ens,'j+;conn')]={}
    key2tf2ratio[(ens,'j-;conn')]={}
    
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    
    mN_jk=ens2pars_jk_meff2st[ens][:,0]
    factor_equal=1/(-3*mN_jk/4)
    
    path=f'{basepath}/conn_2pt.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0])
        
        tf2c2pt={}
        for tf in f['data'].keys():
            t=f[f'data/{tf}'][:]
            t=yu.jackknife(np.real(t[:,:,imom]))
            tf2c2pt[int(tf)]=t

    path=f'{basepath}/conn_0,0,0,0,0,0.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0,0,0,0])
        
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            t=f[f'data/{jtf}'][:]
            t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
            c3pt=yu.jackknife(t)
            ratio=np.real(c3pt/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
            key=(ens,j)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_conn={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;conn')].keys()); tfs.sort()
    ens2tfs_conn[ens]=tfs
    print(ens,tfs)

b [8, 10, 12, 14, 16, 18, 20]
c [6, 8, 10, 12, 14, 16, 18, 20, 22]
d [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]


In [4]:
'''
Symmetrization makes the result slightly different from Christian
'''

key2bare={}

overwrite=False
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    # tfmins_1st=get_tfs(0.5,1.2)
    # tcmins_1st=get_tfs(2*lat_a,0.5)
    # tfmins_2st=get_tfs(0.5,0.8)
    # tcmins_2st=get_tfs(2*lat_a,0.3)
    
    tfmins_2st=range(8,ens2tfs_conn[ens][-1],2); tcmins_2st=range(2,ens2tfs_conn[ens][-1]//2-1)
    
    tf2ratio=key2tf2ratio[(ens,j)]
    # fits_band=yu.doFit_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_fits_band',overwrite=overwrite)
    # fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.8),tf_max=gett(1.3),tcmin=max(gett(0.2),2),corrQ=False)
    # fits_const=yu.doFit_3pt_1st(tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,label=f'{ens}_{j}_fits_const',overwrite=overwrite)
    # fit_const_MA=yu.doMA_3pt(fits_const,tfmin_min=gett(0.8),tcmin_min=3)
    fits_sum=yu.doFit_3pt_sum(tf2ratio,tfmins_2st,tcmins_2st,label=f'{ens}_{j}_fits_sum',overwrite=overwrite)
    # fit_sum_MA=yu.doMA_3pt(fits_sum,tcmin_min=gett(0.2))
    fits_2st=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=ens2pars_jk_meff2st[ens],symmetrizeQ=True,label=f'{ens}_{j}')
    fit_2st_MA=yu.doMA_3pt(fits_2st,tcmin_min=gett(0.2))
    
    key2bare[(ens,j)]=fit_2st_MA[0][:,0]

    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,None,None,fits_sum,fits_2st],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
        # 'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.3),None,None],
        # 'fit_band_WA':fit_band_WA,
        # 'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        # 'fit_const_MA':fit_const_MA,
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,2,5],
        # 'fit_sum_MA':fit_sum_MA,
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,2,5],
        'fit_2st_MA':fit_2st_MA,
        'fit_2st_rainbow_midpoint:[pars_jk_meff2st]':ens2pars_jk_meff2st[ens],
        'xunit':yu.ens2a[ens],
    }
    return dic

js_plt=['j+;conn','j-;conn']; enss_plt=enss

for ij,j in enumerate(js_plt):
    print(f'{ij}/{len(js_plt)}',j,end='                 \r')
    keys=[(ens,j) for ens in enss_plt]

    list_dic=[createDic(key) for key in keys]

    fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','midpoint','fit_2st','fit_2st_prob','fit_sum'])
    yu.addRowHeader(axs,[yu.ens2label[ens] for ens in enss_plt])
    yu.finalizePlot(j)
    
yu.save_pkl_reg('key2bare_conn',key2bare)

# disc

In [5]:
ens2c2pt={}; ens2moms_2pt={}; ens2c2pt0={}; ens2Njk={}
for ens in enss:
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    path=f'{basepath}disc_2pt.h5'
    with h5py.File(path) as f:
        moms_2pt=yu.moms2list(f['moms'])
        c2pt=yu.jackknife(np.real(f['data/N_N'][:,:,:]))
        
    ens2moms_2pt[ens]=moms_2pt
    ens2c2pt[ens]=c2pt
    ens2c2pt0[ens]=c2pt[:,:,moms_2pt.index([0,0,0])]
    ens2Njk[ens]=len(c2pt)

key2tf2ratio={}
stouts=[5,7,9,10,11,15,20]
js=['j+;disc','js;disc','jc;disc']+[f'jg;stout{stout}' for stout in stouts]
for ens in enss:
    path=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/{yu.ens2full[ens]}/data_merge/disc_0,0,1,0,0,0.h5'
    with h5py.File(path) as f:
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            if j not in js:
                continue        
            key=(ens,j)
            if key not in key2tf2ratio:
                key2tf2ratio[key]={}
            c3pt=yu.jackknife(f['data'][jtf][:,:,0,projs.index('P0'),inserts.index('tz')])
            c2pt=ens2c2pt[ens][:,tf,ens2moms_2pt[ens].index([0,0,1])]
            factor=1/(1j*2*np.pi/yu.ens2NL[ens])
            ratio=np.real(c3pt/c2pt[:,None]*factor)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_disc={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;disc')].keys()); tfs.sort()
    ens2tfs_disc[ens]=tfs
    print(ens,tfs[0],tfs[-1])
    
path='pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

b 2 22
c 2 26
d 2 30


In [6]:
selections=['bandfit_WA','const_MA','sum_MA','2st_MA']
selection2key2bare={selection:{} for selection in selections}

overwrite=False
def createDic(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tfmins_1st=get_tfs(0.5,1.2)
    tcmins_1st=get_tfs(2*lat_a,0.5)
    tfmins_2st=get_tfs(0.5,0.8)
    tcmins_2st=get_tfs(2*lat_a,0.3)
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.5))
    fits_band=yu.doFit_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_fits_band',overwrite=overwrite)
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.8),tf_max=gett(1.3),tcmin=gett(0.2),corrQ=False)
    fits_const=yu.doFit_3pt_1st(tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,label=f'{ens}_{j}_fits_const',overwrite=overwrite)
    fit_const_MA=yu.doMA_3pt(fits_const,tfmin_min=gett(0.8),tcmin_min=gett(0.2))
    fits_sum=yu.doFit_3pt_sum(tf2ratio,tfmins_2st,tcmins_2st,label=f'{ens}_{j}_fits_sum',overwrite=overwrite)
    fit_sum_MA=yu.doMA_3pt(fits_sum,tcmin_min=gett(0.2))
    fits_2st=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=ens2pars_jk_meff2st[ens],symmetrizeQ=True,label=f'{ens}_{j}_fits_2st')
    fit_2st_MA=yu.doMA_3pt(fits_2st,tcmin_min=gett(0.2))
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    selection2key2bare['const_MA'][key]=fit_const_MA[0][:,0]
    selection2key2bare['sum_MA'][key]=fit_sum_MA[0][:,0]
    selection2key2bare['2st_MA'][key]=fit_2st_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,fits_2st],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.3),None,None],
        'fit_band_WA':fit_band_WA,
        'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_const_MA':fit_const_MA,
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_sum_MA':fit_sum_MA,
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_2st_MA':fit_2st_MA,
        'xunit':yu.ens2a[ens],
    }
    return dic

def createDic2(key):
    ens,j=key
    gett=lambda t:round(t/yu.ens2a[ens])
    def get_tfs(tmin,tmax,dt=1):
        return range(gett(tmin),gett(tmax),dt)
    lat_a=yu.ens2a[ens]
    
    tfmins_1st=get_tfs(0.4,1.5)
    tcmins_1st=get_tfs(2*lat_a,0.5)
    tfmins_2st=get_tfs(0.4,1.0)
    tcmins_2st=get_tfs(2*lat_a,0.3)
    
    tf2ratio=yu.cut_tf2ratio(key2tf2ratio[(ens,j)],gett(1.6))
    fits_band=yu.doFit_3pt_band(tf2ratio,tcmins_1st,corrQ=False,label=f'{ens}_{j}_fits_band',overwrite=overwrite)
    fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.8),tf_max=gett(1.3),tcmin=gett(0.2),corrQ=False)
    fits_const=yu.doFit_3pt_1st(tf2ratio,tfmins_1st,tcmins_1st,symmetrizeQ=True,label=f'{ens}_{j}_fits_const',overwrite=overwrite)
    fit_const_MA=yu.doMA_3pt(fits_const,tfmin_min=gett(0.55),tcmin_min=gett(0.2))
    fits_sum=yu.doFit_3pt_sum(tf2ratio,tfmins_2st,tcmins_2st,label=f'{ens}_{j}_fits_sum',overwrite=overwrite)
    fit_sum_MA=yu.doMA_3pt(fits_sum,tcmin_min=gett(0.2))
    fits_2st=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=ens2pars_jk_meff2st[ens],symmetrizeQ=True,label=f'{ens}_{j}_fits_2st')
    fit_2st_MA=yu.doMA_3pt(fits_2st,tcmin_min=gett(0.2))
    
    selection2key2bare['bandfit_WA'][key]=fit_band_WA[0][:,0]
    selection2key2bare['const_MA'][key]=fit_const_MA[0][:,0]
    selection2key2bare['sum_MA'][key]=fit_sum_MA[0][:,0]
    selection2key2bare['2st_MA'][key]=fit_2st_MA[0][:,0]
    
    dic={
        'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,fits_2st],
        'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
        'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max]':[gett(0.5),gett(1.3),None,None],
        'fit_band_WA':fit_band_WA,
        'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_const_MA':fit_const_MA,
        'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_sum_MA':fit_sum_MA,
        'fit_2st:[tfmin_min,tfmin_max,tcmin_min,tcmin_max]':[None,None,None,None],
        'fit_2st_MA':fit_2st_MA,
        'xunit':yu.ens2a[ens],
    }
    return dic

# js_plt=['j+;disc']; enss_plt=['b','c','d']
# js_plt=['jg;stout10']; enss_plt=['b','c','d']
js_plt=js; enss_plt=enss

for ij,j in enumerate(js_plt):
    print(f'{ij}/{len(js_plt)}',j,end='                 \r')
    keys=[(ens,j) for ens in enss_plt]

    list_dic=[createDic2(key) if key[1].startswith('jg') else createDic(key) for key in keys]

    yu.makePlot_3pt(list_dic)
    yu.finalizePlot(j)

yu.save_pkl_reg('selection2key2bare_disc',selection2key2bare)

# combine

In [7]:
# RCs

ens2Njk={ens:len(ens2c2pt[ens]) for ens in enss}

path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo(ens2RCs_me[ens][key],ens2RCs_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]
        
path='/p/project1/ngff/li47/code/glwc2/project2/02_discNJN_1D/data_aux/RCs_pre.pkl'
with open(path,'rb') as f:
    ens2RCs_pre_me=pickle.load(f)
ens2RCs_pre={ens:{} for ens in enss[:1]}
for ens in enss[:1]:
    for key in ens2RCs_pre_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs_pre[ens][key]=yu.jackknife_pseudo(ens2RCs_pre_me[ens][key],ens2RCs_pre_me[ens][f'{key}_err']+1e-10,ens2Njk[ens])[:,0]

In [8]:
key2bare_conn=yu.load_pkl_reg('key2bare_conn')
selection2key2bare_disc=yu.load_pkl_reg('selection2key2bare_disc')

selection2key2phy={}
selection2key2phy_pre={}
for selection in selections:
    key2bare={**key2bare_conn,**selection2key2bare[selection]}
    yum.extendBare_avgx(key2bare)
    key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)
    key2phy_pre=yum.bareRC2phy_avgx_pre(key2bare,ens2RCs,ens2RCs_pre)
    
    selection2key2phy[selection]=key2phy
    selection2key2phy_pre[selection]=key2phy_pre

In [9]:
figs=[]
stouts_plt=[5,7,9,10,11,15,20]
selections=['bandfit_WA','const_MA','sum_MA','2st_MA']
for stout in stouts_plt:
    list_dic=[
        {'key2phy':yum.convert_key2phy_stout(selection2key2phy[selection],stout),
        'key2phy_pre':yum.convert_key2phy_stout(selection2key2phy_pre[selection],stout)}
        for selection in selections
    ]
    fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
    fig.suptitle(f'stout{stout}')
    yu.addColHeader(axs,selections)
    yu.finalizePlot(closeQ=True)
    figs.append(fig)
        
yu.makePDF('a2dep',figs)

figs=[]
for selection in selections:
    list_dic=[
        {'key2phy':yum.convert_key2phy_stout(selection2key2phy[selection],stout),
        'key2phy_pre':yum.convert_key2phy_stout(selection2key2phy_pre[selection],stout)}
        for stout in stouts_plt
    ]
    fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
    fig.suptitle(selection)
    yu.addColHeader(axs,[f'stout{stout}' for stout in stouts])
    yu.finalizePlot(closeQ=True)
    figs.append(fig)
    
yu.makePDF('a2dep_stout',figs)

In [10]:
selection='const_MA'; stout=10
list_dic=[
    {'key2phy':yum.convert_key2phy_stout(selection2key2phy[selection],stout),
    'key2phy_pre':yum.convert_key2phy_stout(selection2key2phy_pre[selection],stout)}
]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
yu.finalizePlot(f'a2dep_{selection}_stout{stout}')

In [11]:
selections=['bandfit_WA','const_MA','sum_MA','2st_MA']
stouts_avg=[9,10,11]
ens='a=#_MA'; js_plt=['jtot','jq','jg','ju','jd','js','jc']
tab=[]
for selection in selections:
    key2phy=selection2key2phy[selection]
    def getj(j):
        y_jk=np.transpose([key2phy[(ens,f'{j};stout{stout}')][:,0] for stout in stouts_avg])
        pars_jk,chi2_jk,Ndof=yu.doFit_const(y_jk)
        return yu.jackme_un2str(pars_jk[:,0])
    
    tab.append([getj(j) for j in js_plt])

df=pd.DataFrame(tab,index=selections,columns=js_plt)
display(df)


,jtot,jq,jg,ju,jd,js,jc
bandfit_WA,0.974(63),0.523(46),0.430(28),0.321(17),0.158(14),0.037(13),8(11)e-3
const_MA,0.893(49),0.525(44),0.379(16),0.316(18),0.153(14),0.040(13),0.015(11)
sum_MA,1.008(69),0.548(54),0.443(30),0.324(19),0.161(16),0.046(15),0.016(13)
2st_MA,1.18(24),0.64(19),0.48(11),0.370(56),0.207(56),0.058(51),3(36)e-3
